In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets
from torch.utils.data import DataLoader, random_split
from torchvision.transforms import v2
from tqdm import tqdm
import warnings

warnings.filterwarnings("ignore")

In [ ]:
class MultilayerPerceptron(nn.Module):
    def __init__(self, in_features, out_features):
        super(MultilayerPerceptron, self).__init__()
        self.layers = nn.Sequential(
            nn.Flatten(),
            nn.Linear(in_features, 32),
            nn.ReLU(),
            nn.Linear(32, out_features),
        )

    def forward(self, x):
        return self.layers(x)

In [ ]:
BATCH_SIZE = 64
LEARNING_RATE = 1e-3
EPOCHS = 5
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
transformations = v2.Compose(
    (
        v2.ToTensor(),
    )
)

net = MultilayerPerceptron(784, 10).to(DEVICE)

dataset = datasets.MNIST("../data", True, transformations, download=True)
train_dataset, test_dataset = random_split(dataset, [0.8, 0.2])
dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
optimizer = optim.SGD(net.parameters(), LEARNING_RATE)
criterion = nn.CrossEntropyLoss()

In [ ]:
net.train()

for epoch in range(EPOCHS):
    for images, labels in tqdm(dataloader, desc=f"Epoch {epoch + 1}"):
        images = images.to(DEVICE)
        labels = labels.to(DEVICE)

        net.zero_grad()

        predictions = net(images)
        loss = criterion(predictions, labels)

        loss.backward()
        optimizer.step()
        
    print(f"Epoch: {epoch + 1} Loss: {loss}")

In [ ]:
net.eval()
correct = 0

for image, label in test_dataset:
    prediction = torch.argmax(F.softmax(net(image)))
    
    if prediction == label:
        correct += 1

print(f"Accuracy: {correct / len(test_dataset) * 100}%")